In [66]:
workspace()
import Base: getindex, setindex!

abstract Frame

function getindex(frame::Frame, name::Symbol)
    frame[getCoordinateIndex(frame, name)]
end

function getindex(frame::Frame, index::Integer)
    frame.data[index]
end

function setindex!(frame::Frame, value, name::Symbol)
    frame[getCoordinateIndex(frame, name)] = value
end

function setindex!(frame::Frame, value, index::Integer)
    frame.data[index] = value
end

type VectorFrame <: Frame
    data::Vector
    coordinate_names::Vector{Symbol}
    coordinate_map::Dict{Symbol, Integer}
    
    VectorFrame(data::Vector, coordinate_names::Vector{Symbol}) = new(data, coordinate_names, Dict{Symbol,Integer}(zip(coordinate_names, 1:length(coordinate_names))))
end

function getCoordinateIndex(frame::VectorFrame, name::Symbol)
    frame.coordinate_map[name]
#     find(name .== frame.coordinate_names)[1]
end

abstract System

type DoubleIntegrator <: System
end

type DoubleIntegratorState <: Frame
    z
    zdot
end

function getindex(frame::DoubleIntegratorState, name::Symbol)
    getfield(frame, name)
end

function setindex!(frame::DoubleIntegratorState, value, name::Symbol)
    setfield!(frame, name, value)
end

function dynamics!(xdot::Frame, sys::DoubleIntegrator, t::Real, x::Frame, u::Frame)
#     xdot = VectorFrame(x.data, x.coordinate_names)
    xdot[:z] = x[:zdot]
    xdot[:zdot] = u[:z]
end
    


dynamics! (generic function with 1 method)

In [71]:
sys = DoubleIntegrator()
x = VectorFrame([0.0, 1.0], [:z, :zdot])
u = VectorFrame([2.0], [:z])

xdot = VectorFrame(copy(x.data), x.coordinate_names)
dynamics!(xdot, sys, 0, x, u)
@time for j = 1:1e3; dynamics!(xdot, sys, 0, x, u); end
xdot

VectorFrame([1.0,2.0],[:z,:zdot],Dict{Symbol,Integer}(:z=>1,:zdot=>2))

0.000293 seconds (2.00 k allocations: 31.250 KB)


In [72]:
sys = DoubleIntegrator()
x = DoubleIntegratorState(0.0, 1.0)
u = VectorFrame([2.0], [:z])

xdot = DoubleIntegratorState(0.0, 0.0)
dynamics!(xdot, sys, 0, x, u)
@time for j = 1:1e3; dynamics!(xdot, sys, 0, x, u); end
xdot

DoubleIntegratorState(1.0,2.0)

0.000108 seconds (1000 allocations: 15.625 KB)
